<a href="https://colab.research.google.com/github/MrDog1/terms-of-service/blob/main/PathoBERT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PathologyBERT Google Colab デモ
# 必要なライブラリをインストール
!pip install transformers -q

import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
from google.colab import widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd

# タイトルとイントロダクション表示
print("="*80)
print("PathologyBERT デモ")
print("="*80)
print("このノートブックでは、病理学に特化した事前学習モデル「PathologyBERT」を試すことができます。")
print("テキスト内の単語を「[MASK]」でマスクすると、モデルがその部分に適切な単語を予測します。")
print("="*80)

# モデルとトークナイザーの読み込み
print("モデルを読み込んでいます...")
try:
    tokenizer = AutoTokenizer.from_pretrained("tsantos/PathologyBERT")
    model = AutoModelForMaskedLM.from_pretrained("tsantos/PathologyBERT")
    nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
    print("モデルの読み込みが完了しました。")
except Exception as e:
    print(f"モデルの読み込みエラー: {e}")

# サンプル例
examples = [
    "intraductal papilloma with [MASK] and micro calcifications",
    "micro calcifications with usual ductal hyperplasia and no [MASK] lesions identified.",
    "invasive [MASK] carcinoma, grade 2, measuring 1.5 cm in greatest dimension.",
    "the lymph nodes are negative for [MASK] involvement.",
    "specimen shows ductal carcinoma in [MASK] (DCIS), intermediate nuclear grade."
]

# 予測関数の定義
def predict(text):
    if "[MASK]" not in text:
        print("警告: テキスト内に '[MASK]' を含めてください")
        return

    try:
        results = nlp(text)

        # 結果の表示
        print("\n予測結果:")

        # データフレームで結果を表示
        result_data = {
            "予測単語": [],
            "スコア": [],
            "予測テキスト": []
        }

        for result in results:
            result_data["予測単語"].append(result["token_str"])
            result_data["スコア"].append(f"{result['score']:.4f}")
            result_data["予測テキスト"].append(result["sequence"].replace("[CLS] ", "").replace(" [SEP]", ""))

        df = pd.DataFrame(result_data)
        display(df)

    except Exception as e:
        print(f"予測エラー: {e}")

# 対話型ウィジェットの作成
dropdown = widgets.Dropdown(
    options=examples,
    description='サンプル例:',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

text_input = widgets.Textarea(
    value=examples[0],
    placeholder='[MASK]を含むテキストを入力',
    description='テキスト:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

button = widgets.Button(
    description='予測実行',
    disabled=False,
    button_style='primary',
    tooltip='クリックして予測を実行',
    icon='check'
)

output = widgets.Output()

# ドロップダウンの値変更時の処理
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        text_input.value = change['new']

dropdown.observe(on_dropdown_change)

# ボタンクリック時の処理
def on_button_clicked(b):
    with output:
        clear_output()
        predict(text_input.value)

button.on_click(on_button_clicked)

# ウィジェットの表示
print("\nサンプル例から選択するか、カスタムテキストを入力してください:")
display(dropdown)
display(text_input)
display(button)
display(output)

# 初回実行例
with output:
    predict(examples[0])

print("\n"+"="*80)
print("PathologyBERTについて")
print("="*80)
print("PathologyBERTは病理組織標本レポートで訓練された事前学習済みマスク言語モデルです。")
print("一般的なBERTモデルとは異なり、病理学に特化した語彙を持ち、特に乳腺病理に関する")
print("専門用語の理解に優れています。")
print("\n詳細については、開発者のGitHubリポジトリを参照してください:")
print("https://github.com/thiagosantos1/HCSBC")
print("="*80)

# ボーナス: モデルの語彙を探索する関数
def explore_vocab(query, top_n=10):
    """モデルの語彙から特定のクエリに一致する単語を検索"""
    matches = []
    for i in range(len(tokenizer.vocab)):
        token = tokenizer.convert_ids_to_tokens(i)
        if query.lower() in token.lower() and not token.startswith('##'):
            matches.append(token)

    print(f"'{query}'に一致する語彙例 (上位{min(top_n, len(matches))}件):")
    for i, token in enumerate(matches[:top_n]):
        print(f"  {i+1}. {token}")

    return matches

# 使用例
print("\n病理学の専門用語を探索:")
explore_vocab("carcin", 10)  # "carcinoma"関連の単語を探索

PathologyBERT デモ
このノートブックでは、病理学に特化した事前学習モデル「PathologyBERT」を試すことができます。
テキスト内の単語を「[MASK]」でマスクすると、モデルがその部分に適切な単語を予測します。
モデルを読み込んでいます...


Device set to use cuda:0


モデルの読み込みが完了しました。

サンプル例から選択するか、カスタムテキストを入力してください:


Dropdown(description='サンプル例:', layout=Layout(width='100%'), options=('intraductal papilloma with [MASK] and mi…

Textarea(value='intraductal papilloma with [MASK] and micro calcifications', description='テキスト:', layout=Layou…

Button(button_style='primary', description='予測実行', icon='check', style=ButtonStyle(), tooltip='クリックして予測を実行')

Output()


PathologyBERTについて
PathologyBERTは病理組織標本レポートで訓練された事前学習済みマスク言語モデルです。
一般的なBERTモデルとは異なり、病理学に特化した語彙を持ち、特に乳腺病理に関する
専門用語の理解に優れています。

詳細については、開発者のGitHubリポジトリを参照してください:
https://github.com/thiagosantos1/HCSBC

病理学の専門用語を探索:
'carcin'に一致する語彙例 (上位10件):
  1. carcinoma
  2. adenocarcinoma
  3. carcinomas
  4. carcin
  5. carcinoid
  6. microcarcinoma
  7. carcinosarcoma
  8. comedocarcinoma
  9. cholangiocarcinoma
  10. carcinom


['carcinoma',
 'adenocarcinoma',
 'carcinomas',
 'carcin',
 'carcinoid',
 'microcarcinoma',
 'carcinosarcoma',
 'comedocarcinoma',
 'cholangiocarcinoma',
 'carcinom',
 'carcinomatous',
 'microcarcinomas']